# Flight Data Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [75]:
from pyarrow.flight import FlightClient, Ticket, FlightCallOptions
import json
import time
from flightsql import FlightSQLClient
from IPython.display import display, HTML

The class `query_timer` provides a simple stop watch function for query performance testing. This will be utilised later within the notebook.

In [76]:
class query_timer:
    def __init__(self):
        self.start_time = time.time()
        self.end_time = None
        self.elapsed_time = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        self.elapsed_time = self.end_time - self.start_time
        print('Execution time:', self.elapsed_time, 'seconds')
        return self.elapsed_time


Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [77]:
token = ''
host = 'host'    
client = FlightClient(f"grpc+tls://{host}:443")
  
    
    # create an authorization header
options = FlightCallOptions(headers=[(b"authorization",f"Bearer {token}".encode('utf-8'))])

The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

In [78]:
# This functionality will be wrapped in the Python client library, 
# but it's good to see how it works
def query_tools(database="", query="", language="sql"):
    # create a flight client pointing to the InfluxDB
    # create a ticket
    ticket_data = {
    "database": database,
    "sql_query": query,
    "query_type": language}
    
    ticket_bytes = json.dumps(ticket_data)
    ticket = Ticket(ticket_bytes)
    
    # execute the query and return all the data
    flight_reader = client.do_get(ticket, options)

    # use read_all() to get all of the data as an Arrow table
    # there are other functions to iterate through rows or read only parts of the data
    # which is useful if you have huge data sets
    return flight_reader.read_all()

Return all columns from the table flight for the past 3 days.

In [79]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT * FROM \"flight\" WHERE time >= now() - interval '1 hour'")
df = table.to_pandas()
display(df)

#TODO add number of rows returned if required.

Execution time: 2.088702917098999 seconds


,baro_altitude,callsign,category,geo_altitude,icao24,last_contact,latitude,longitude,on_ground,origin_country,position_source,sensors,spi,squawk,time,time_position,true_track,velocity,vertical_rate
0,4305.3,,0,233.00,d01073,1684762955,51.4734,6.4363,False,<NA>,3,None,False,0142,2023-05-22 14:42:43.528557,1684762955,0.00,0.00,0.0
1,4305.3,,0,233.00,d01073,1684762955,51.4734,6.4363,False,<NA>,3,None,False,0142,2023-05-22 14:43:09.460129,1684762955,0.00,0.00,0.0
2,4305.3,,0,233.00,d01073,1684762955,51.4734,6.4363,False,<NA>,3,None,False,0142,2023-05-22 14:43:35.215190,1684762955,0.00,0.00,0.0
3,4305.3,,0,233.00,d01073,1684762955,51.4734,6.4363,False,<NA>,3,None,False,0142,2023-05-22 14:44:00.959938,1684762955,0.00,0.00,0.0
4,4305.3,,0,233.00,d01073,1684762955,51.4734,6.4363,False,<NA>,3,None,False,0142,2023-05-22 14:44:26.678951,1684762955,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87595,12192.0,VJC895,0,12969.24,8881db,1684762955,14.1460,100.4487,False,Viet Nam,0,None,False,7464,2023-05-22 14:44:26.678951,1684762954,304.58,249.29,0.0
87596,12192.0,VJC895,0,12969.24,8881db,1684762955,14.1460,100.4487,False,Viet Nam,0,None,False,7464,2023-05-22 14:44:52.456026,1684762954,304.58,249.29,0.0
87597,12192.0,VJC895,0,12969.24,8881db,1684762955,14.1460,100.4487,False,Viet Nam,0,None,False,7464,2023-05-22 14:45:21.733319,1684762954,304.58,249.29,0.0
87598,12192.0,VJC895,0,12969.24,8881db,1684762955,14.1460,100.4487,False,Viet Nam,0,None,False,7464,2023-05-22 14:46:16.234436,1684762954,304.58,249.29,0.0


Count the number of unique tags

In [80]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT count(distinct(icao24)) as \"icao24\", count(distinct(origin_country)) as \"origin_country\" FROM \"flight\" WHERE time >= now() - interval '3 day'")
df = table.to_pandas()
display(df)

Execution time: 0.18212080001831055 seconds


,icao24,origin_country
0,15864,107


# InfluxQL based query

In [81]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT * FROM flight WHERE time > now() - 1h", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 1.1558139324188232 seconds


,iox::measurement,time,baro_altitude,callsign,category,geo_altitude,icao24,last_contact,latitude,longitude,on_ground,origin_country,position_source,sensors,spi,squawk,time_position,true_track,velocity,vertical_rate
0,flight,2023-05-22 14:42:43.528557,99.06,LNK664,4,167.64,008081,1684762955,-33.9487,18.5965,False,South Africa,0,None,False,4316,1684762954,165.38,61.14,-3.58
1,flight,2023-05-22 14:42:43.528557,723.90,ZSHIE,0,762.00,00830b,1684762824,-33.9377,18.3528,False,South Africa,0,None,False,1000,1684762824,188.50,55.66,0.00
2,flight,2023-05-22 14:42:43.528557,1744.98,LNK151,0,1859.28,008b8f,1684762920,-26.1877,28.2410,False,South Africa,0,None,False,0545,1684762917,14.66,69.13,-2.60
3,flight,2023-05-22 14:42:43.528557,1691.64,SFR345,0,1859.28,008de1,1684762698,-26.1777,28.2441,False,South Africa,0,None,False,3402,1684762698,15.64,80.13,-4.55
4,flight,2023-05-22 14:42:43.528557,2141.22,SFR131,0,2225.04,008de4,1684762787,-33.9663,18.6021,True,South Africa,0,None,False,3352,1684762787,165.94,64.31,-6.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87595,flight,2023-05-22 14:46:43.567914,3276.60,JAT403,4,12519.66,e80600,1684762954,-12.2816,-77.2686,False,Chile,0,None,False,6756,1684762954,166.05,160.08,5.20
87596,flight,2023-05-22 14:46:43.567914,7406.64,JAP7319,0,8564.88,e80609,1684762955,-10.7077,-77.6929,False,Chile,0,None,False,1055,1684762955,166.99,187.44,-8.78
87597,flight,2023-05-22 14:46:43.567914,5699.76,JAP7000,0,8061.96,e80610,1684762954,-12.6246,-76.8993,False,Chile,0,None,False,6360,1684762954,312.94,191.83,-10.73
87598,flight,2023-05-22 14:46:43.567914,3040.38,LNE1360,1,3246.12,e84072,1684762955,0.0355,-78.4121,False,Ecuador,0,None,False,0571,1684762955,140.93,102.04,0.00


# Schema Navigation

In this section you can find some examples for schema navigation. 

In [82]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SHOW FIELD KEYS", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 0.10384702682495117 seconds


,iox::measurement,fieldKey,fieldType
0,flight,baro_altitude,float
1,flight,callsign,string
2,flight,category,integer
3,flight,geo_altitude,float
4,flight,last_contact,integer
5,flight,latitude,float
6,flight,longitude,float
7,flight,on_ground,boolean
8,flight,position_source,integer
9,flight,sensors,string


In [83]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SHOW TAG KEYS", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 0.10353779792785645 seconds


,iox::measurement,tagKey
0,flight,icao24
1,flight,origin_country
